In [1]:
import csv
from heapq import heapify, heappush, heappop

In [2]:
class MempoolTransaction():
    """Save each transaction as an object"""
    def __init__(self, txid, fee, weight, parents):
        self.txid = txid
        self.fee = int(fee)
        self.weight= int(weight)
        if len(parents)==0:
            self.parents =[]
        else:
            self.parents= parents.split('\n')[0].split(';')

In [3]:
def parse_mempool_csv():
    """Parse the CSV file and return a list of MempoolTransactions."""
    data= open('mempool.csv', 'r').readlines()[1:]
    return([MempoolTransaction(*line.strip().split(',')) for line in data])

In [4]:
class HeapSolution():
    
    def __init__(self):
        self.transactions= parse_mempool_csv()    
        self.heap= self.createHeap()
        self.blockWeight =0
        self.visited= set()
        self.blockTransactions= []
        self.suspendedTransactions= []
        
        self.get_appropriateTransactions()

    def createHeap(self,):
        heap = [] 
        heapify(heap) 

        for entry in self.transactions:
            heappush(heap, (-1*entry.fee, entry.txid, entry.weight, entry.parents))

        return heap
    
    def pull_suspendedTransactions(self,):
        for entry in self.suspendedTransactions:
            heappush(self.heap, (entry.fee, entry.txid, entry.weight, entry.parents))    
    
    def get_appropriateTransactions(self, ):
        while self.heap and self.blockWeight <=400000:
            entry= heappop(self.heap)
            fee, txid, weight, parents= -1*entry[0], entry[1], entry[2], entry[3]
            if len(parents) ==0:
#                 print ("No Parents")
                self.blockTransactions.append((fee, weight, txid))
                self.visited.add(txid)
                self.blockWeight +=weight
                self.pull_suspendedTransactions()

            else:
                isSuspended= False
                for parent in parents:
                    if parent not in self.visited:
#                         print ("Parent unvisited")
                        self.suspendedTransactions.append(entry)
                        isSuspended= True
                        continue
                if not isSuspended:
#                     print ("All parent visited")
                    self.blockTransactions.append((fee, weight, txid))
                    self.visited.add(txid)
                    self.blockWeight +=weight
                    self.pull_suspendedTransactions()
        
        profit =0
        for row in self.blockTransactions:
            profit += row[0]
            
        print (profit)

In [5]:
class KnapsackSolution():
    def __init__(self):
        self.transactions= parse_mempool_csv()    
        profit= self.get_appropriateTransactions(40000, len(self.transactions))
        print (profit)
        
    def get_appropriateTransactions(self, capacity, n):
        
        dp = [0 for i in range(capacity+1)]
 
        for i in range(1, n+1):  
            for w in range(capacity, 0, -1):  
                if self.transactions[i-1].weight <= w:
                    dp[w] = max(dp[w], dp[w-self.transactions[i-1].weight]+self.transactions[i-1].fee)

        return dp[capacity]  

In [6]:
if __name__ == "__main__":
    HeapSolution()
    KnapsackSolution()

1263020
1417260
